# Download and Process Supplementary Table S2 (BigFIT Paper)

**Purpose**: Download curated growth/no-growth data from BigFIT paper Supplementary Tables

**Data Source**: Price et al. (2018) Nature 557:503-509
- Paper: https://www.nature.com/articles/s41586-018-0124-0
- Supplementary Tables: https://genomics.lbl.gov/supplemental/bigfit/Supplementary_Tables_final.xlsx

**What is Supplementary Table S2?**
- Gold standard growth/no-growth data for 28 organisms
- 95 carbon sources tested (plus water control)
- TRUE = organism grows on carbon source
- FALSE = organism does NOT grow on carbon source
- Conservative threshold - "No Growth" may grow under different conditions

**Output Files**:
- `results/supplementary_table_s2_clean.csv` - Growth matrix (organisms × carbon sources)
- `results/supplementary_table_s2_carbon.csv` - Same data, transposed view
- `results/organism_name_mapping_supplementary_to_FIT.csv` - Maps organism names to Fitness Browser orgIds

**Last updated**: 2025-10-06

## Setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import urllib.request
import sqlite3
import re

print("Imports successful")

## Configuration

In [ ]:
# URLs
SUPP_TABLE_URL = "https://genomics.lbl.gov/supplemental/bigfit/Supplementary_Tables_final.xlsx"

# Paths
DOWNLOADS_DIR = Path("../downloads")
DOWNLOADS_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_DIR = Path("results")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

EXCEL_FILE = DOWNLOADS_DIR / "Supplementary_Tables_final.xlsx"
DB_PATH = DOWNLOADS_DIR / "feba.db"

# Output files
OUTPUT_CLEAN = OUTPUT_DIR / "supplementary_table_s2_clean.csv"
OUTPUT_CARBON = OUTPUT_DIR / "supplementary_table_s2_carbon.csv"
OUTPUT_MAPPING = OUTPUT_DIR / "organism_name_mapping_supplementary_to_FIT.csv"

print(f"Configuration set")
print(f"  Download URL: {SUPP_TABLE_URL}")
print(f"  Output directory: {OUTPUT_DIR}")

## Download Supplementary Tables

In [ ]:
if EXCEL_FILE.exists():
    print(f"Supplementary Tables already downloaded: {EXCEL_FILE}")
    print(f"  Size: {EXCEL_FILE.stat().st_size / 1e6:.2f} MB")
else:
    print(f"Downloading Supplementary Tables from: {SUPP_TABLE_URL}")
    urllib.request.urlretrieve(SUPP_TABLE_URL, EXCEL_FILE)
    print(f"Downloaded to: {EXCEL_FILE}")
    print(f"  Size: {EXCEL_FILE.stat().st_size / 1e6:.2f} MB")

## Read Table S2 (Carbon Sources)

In [ ]:
# Table S2 starts at row 40 (0-indexed row 39)
# Row 40 has headers, data starts at row 41
print("Reading Supplementary Table S2 (Carbon sources)...")

df_s2 = pd.read_excel(
    EXCEL_FILE,
    sheet_name='Table S2',
    header=40  # Row 41 in Excel (0-indexed)
)

print(f"\nLoaded Table S2")
print(f"  Shape: {df_s2.shape}")
print(f"  Columns: {len(df_s2.columns)}")
print(f"\nFirst few columns: {list(df_s2.columns[:10])}")

## Extract Carbon Source Growth Data

In [ ]:
# First 4 columns are metadata, organisms start at column 5 (index 4)
# Find where organism columns start
organism_start_col = 4  # Column E in Excel

# Extract carbon sources (first column) and organism growth data
carbon_sources = df_s2.iloc[:, 0].dropna()  # First column = carbon source names
organism_data = df_s2.iloc[:, organism_start_col:]  # Columns E onwards = organisms

print(f"Extracted data:")
print(f"  Carbon sources: {len(carbon_sources)}")
print(f"  Organism columns: {len(organism_data.columns)}")
print(f"\nFirst 5 carbon sources:")
for i, cs in enumerate(carbon_sources.head()):
    print(f"  {i+1}. {cs}")

## Clean and Format Data

In [ ]:
# Create clean dataframe with carbon sources as rows, organisms as columns
growth_data = organism_data.copy()
growth_data.index = carbon_sources

# Remove water control (not a carbon source)
if 'Water' in growth_data.index:
    print("Removing water control...")
    growth_data = growth_data.drop('Water')

# Convert TRUE/FALSE to Growth/No Growth
growth_data = growth_data.replace({True: 'Growth', False: 'No Growth'})

print(f"\nCleaned growth matrix:")
print(f"  Shape: {growth_data.shape[0]} carbon sources × {growth_data.shape[1]} organisms")
print(f"  Growth calls: {(growth_data == 'Growth').sum().sum()}")
print(f"  No Growth calls: {(growth_data == 'No Growth').sum().sum()}")

## Get Organism Names and Map to Fitness Browser orgIds

In [ ]:
# Get organism names from column headers
supp_organism_names = growth_data.columns.tolist()

print(f"Organism names in Supplementary Table S2:")
for i, name in enumerate(supp_organism_names, 1):
    print(f"  {i:2d}. {name}")

In [ ]:
# Load organism metadata from Fitness Browser database
if not DB_PATH.exists():
    print(f"WARNING: Fitness Browser database not found at {DB_PATH}")
    print(f"Cannot create organism mapping. Please download feba.db first.")
    organism_mapping = None
else:
    print(f"Loading organism data from Fitness Browser database...")
    conn = sqlite3.connect(str(DB_PATH))
    
    query = """
    SELECT orgId, genus, species, strain
    FROM Organism
    ORDER BY orgId
    """
    fb_organisms = pd.read_sql_query(query, conn)
    conn.close()
    
    # Create full species name
    fb_organisms['full_name'] = (
        fb_organisms['genus'] + ' ' + 
        fb_organisms['species'] + ' ' + 
        fb_organisms['strain']
    ).str.strip()
    
    print(f"Loaded {len(fb_organisms)} organisms from Fitness Browser")
    
    # Create mapping by fuzzy matching
    # Manual mapping based on known organism names
    organism_mapping = {
        'Acidovorax sp. GW101-3H11': 'acidovorax_3H11',
        'Azospirillum brasilense sp. 245': 'azobra',
        'Burkholderia phytofirmans PsJN': 'BFirm',
        'Caulobacter crescentus NA1000': 'Caulo',
        'Cupriavidus basilensis 4G11': 'Cup4G11',
        'Dechlorosoma suillum PS': 'PS',
        'Dinoroseobacter shibae DFL-12': 'Dino',
        'Dyella japonica UNC79MFTsu3.2': 'Dyella79',
        'Echinicola vietnamensis DSM 17526': 'Cola',
        'Escherichia coli BW25113': 'Keio',
        'Herbaspirillum seropedicae SmR1': 'HerbieS',
        'Kangiella aquimarina DSM 16071': 'Kang',
        'Marinobacter adhaerens HP15': 'Marino',
        'Pedobacter sp. GW460-11-11-14-LB5': 'Pedo557',
        'Phaeobacter inhibens BS107': 'Phaeo',
        'Pseudomonas fluorescens FW300-N1B4': 'pseudo1_N1B4',
        'Pseudomonas fluorescens FW300-N2C3': 'pseudo5_N2C3_1',
        'Pseudomonas fluorescens FW300-N2E2': 'pseudo6_N2E2',
        'Pseudomonas fluorescens FW300-N2E3': 'pseudo3_N2E3',
        'Pseudomonas fluorescens GW456-L13': 'pseudo13_GW456_L13',
        'Pseudomonas simiae WCS417': 'WCS417',
        'Pseudomonas stutzeri RCH2': 'psRCH2',
        'Shewanella amazonensis SB2B': 'SB2B',
        'Shewanella loihica PV-4': 'PV4',
        'Shewanella oneidensis MR-1': 'MR1',
        'Shewanella sp. ANA-3': 'ANA3',
        'Sinorhizobium meliloti 1021': 'Smeli',
        'Sphingomonas koreensis DSMZ 15582': 'Korea'
    }
    
    print(f"\nCreated mapping for {len(organism_mapping)} organisms")

## Rename Columns to Simple Species Names

In [ ]:
# Rename columns to simple species names (no division prefix)
def clean_species_name(name):
    """Remove division prefix from species name"""
    if ': ' in name:
        return name.split(': ', 1)[1]
    return name

growth_data.columns = [clean_species_name(col) for col in growth_data.columns]

print("Renamed columns to simple species names:")
for col in growth_data.columns:
    print(f"  {col}")

## Save Cleaned Data

In [ ]:
# Save main growth matrix (carbon sources as rows)
print(f"Saving cleaned growth matrix to: {OUTPUT_CLEAN}")
growth_data.to_csv(OUTPUT_CLEAN)
print(f"Saved {growth_data.shape[0]} carbon sources × {growth_data.shape[1]} organisms")

# Save transposed version (organisms as rows) for easier viewing
print(f"\nSaving transposed matrix to: {OUTPUT_CARBON}")
growth_data_transposed = growth_data.T
growth_data_transposed.to_csv(OUTPUT_CARBON)
print(f"Saved {growth_data_transposed.shape[0]} organisms × {growth_data_transposed.shape[1]} carbon sources")

# Save organism mapping
if organism_mapping:
    print(f"\nSaving organism mapping to: {OUTPUT_MAPPING}")
    mapping_df = pd.DataFrame(
        list(organism_mapping.items()),
        columns=['Supplementary_Table_Name', 'Fitness_Browser_OrgID']
    )
    mapping_df.to_csv(OUTPUT_MAPPING, index=False)
    print(f"Saved {len(mapping_df)} organism mappings")

## Summary Statistics

In [ ]:
print("="*70)
print("SUPPLEMENTARY TABLE S2 SUMMARY")
print("="*70)

print(f"\nData source:")
print(f"  Paper: Price et al. (2018) Nature 557:503-509")
print(f"  URL: {SUPP_TABLE_URL}")

print(f"\nMatrix dimensions:")
print(f"  Organisms: {growth_data.shape[1]}")
print(f"  Carbon sources: {growth_data.shape[0]}")
print(f"  Total comparisons: {growth_data.size:,}")

print(f"\nGrowth calls:")
n_growth = (growth_data == 'Growth').sum().sum()
n_no_growth = (growth_data == 'No Growth').sum().sum()
print(f"  Growth: {n_growth:,} ({100*n_growth/growth_data.size:.1f}%)")
print(f"  No Growth: {n_no_growth:,} ({100*n_no_growth/growth_data.size:.1f}%)")

print(f"\nFiles created:")
print(f"  {OUTPUT_CLEAN}")
print(f"  {OUTPUT_CARBON}")
if organism_mapping:
    print(f"  {OUTPUT_MAPPING}")

print(f"\nInterpretation:")
print(f"  'Growth' = Organism confirmed to grow on carbon source")
print(f"  'No Growth' = Organism did NOT grow (conservative threshold)")
print(f"  Note: 'No Growth' may grow under different conditions")

print("\n" + "="*70)